# Min Max area - Regresor by Xgboost

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import math

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.constrained_layout.use'] = True
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data_tmp = dataset.copy()
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]
def bar14h(tick):
    tick=tick[(tick.index.hour == 14) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]
data_tmp['Close_9h'] = data_tmp['Close']
data_tmp['Close_14h'] = data_tmp['Close']
day_data = data_tmp.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h,
        'Close_14h': bar14h
    })
day_data.dropna(subset=['Close'], inplace=True)
day_data['Close_shift'] = day_data['Close'].shift(1)
day_data['Close_14h_shift'] = day_data['Close_14h'].shift(1)
day_data['Day_High_shift'] = day_data['High'].shift(1)
day_data['Day_Low_shift'] = day_data['Low'].shift(1)
day_data = day_data[['Close', 'High', 'Low', 'Close_shift', 'Close_14h_shift', 'Day_High_shift', 'Day_Low_shift']].rename(columns={'Close': 'Day_Close', 'High': 'Day_High', 'Low': 'Day_Low'})

In [4]:
day_data

,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift
Date,,,,,,,
2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN
2018-08-14,959.3,961.8,952.1,954.2,949.5,954.6,942.3
2018-08-15,946.6,962.3,946.6,959.3,960.8,961.8,952.1
2018-08-16,947.0,947.3,935.5,946.6,957.7,962.3,946.6
2018-08-17,947.0,954.6,945.0,947.0,944.0,947.3,935.5
...,...,...,...,...,...,...,...
2024-07-29,1287.8,1294.7,1287.8,1285.0,1278.4,1285.0,1276.8
2024-07-30,1292.4,1292.4,1284.2,1287.8,1292.9,1294.7,1287.8
2024-07-31,1304.3,1304.3,1291.1,1292.4,1284.9,1292.4,1284.2


In [5]:
data = dataset.copy()
data = data.assign(time_d=pd.PeriodIndex(data.index, freq='1D').to_timestamp())
data = pd.merge(data, day_data, left_on="time_d", right_index=True, how="left")

In [6]:
data = data[data.High != data.Low]
def get_percent(r):
    return (r['Close'] - r['Day_Low']) / (r['Day_High'] - r['Day_Low'])
data['percent'] = data.apply(lambda r: get_percent(r), axis=1)

In [7]:
data

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent
Date,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.065041
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.081301
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.065041
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.024390
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.113821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,22701,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.683962
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,12412,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.702830
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.867925


In [8]:
data['last_close_rate'] = 1000*(data.Close - data.Close_shift) / data.Close_shift
data['last_highlow_rate'] = (data.Close - data.Close_14h_shift) /  (data.Day_High_shift - data.Day_Low_shift)

In [9]:
data.dropna(inplace=True)

In [10]:
data_test = data.copy()
data_test = data_test[data_test.index > '2024-01-01 00:00:00']
### Ignore this year data => tranh over fitting
data = data[data.index < '2024-01-01 00:00:00']

In [11]:
data = data.drop(data[(data.percent > 0.2) & (data.percent < 0.8)].sample(frac=.75, random_state=1).index)

In [12]:
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE

In [13]:
feature_cols = ["last_close_rate", "last_highlow_rate"]
# Splitting 
X_train = data[feature_cols]
y_train = data[['percent']]
X_test = data_test[feature_cols]
y_test = data_test[['percent']]
  
# Instantiation 
xgb_r = xg.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 100, seed = 123)
  
# Fitting the model 
xgb_r.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [14]:
# Predict the model 
y_pred = xgb_r.predict(X_test) 
  
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, y_pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.245469


In [15]:
X_result = X_test.copy()
X_result = X_result.assign(Predicts=y_pred)
X_result = X_result[['Predicts']]
data_test = pd.merge(data_test, X_result, left_index=True, right_index=True, how="left")
data_test.dropna(inplace=True)

In [16]:
data_test

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-02 09:00:00,1138.5,1139.7,1138.5,1138.9,6206,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.731481,3.789882,0.813953,0.543117
2024-01-02 09:05:00,1138.9,1139.0,1138.6,1139.0,1424,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.740741,3.878019,0.825581,0.616176
2024-01-02 09:10:00,1139.0,1141.5,1138.9,1141.4,6265,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.962963,5.993302,1.104651,0.617882
2024-01-02 09:15:00,1141.3,1141.8,1140.3,1140.8,3739,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.907407,5.464481,1.034884,0.784172
2024-01-02 09:20:00,1140.7,1141.2,1140.2,1140.2,1943,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.851852,4.935660,0.965116,0.587992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,22701,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.683962,-3.294118,-0.360795,0.387391
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,12412,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.702830,-2.980392,-0.349432,0.471103
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.867925,-0.235294,-0.250000,0.482568


### No signal for a Short

In [17]:
data_test[(data_test.percent > 0.8) & (data_test.Predicts < 0.8)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-02 09:10:00,1139.0,1141.5,1138.9,1141.4,6265,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.962963,5.993302,1.104651,0.617882
2024-01-02 09:15:00,1141.3,1141.8,1140.3,1140.8,3739,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.907407,5.464481,1.034884,0.784172
2024-01-02 09:20:00,1140.7,1141.2,1140.2,1140.2,1943,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.851852,4.935660,0.965116,0.587992
2024-01-02 09:25:00,1140.2,1140.5,1139.6,1139.7,3265,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.805556,4.494976,0.906977,0.594105
2024-01-02 09:30:00,1139.9,1140.0,1139.6,1139.9,2584,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.824074,4.671250,0.930233,0.533563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01 11:10:00,1299.7,1300.2,1299.0,1299.6,1964,2024-08-01,1275.0,1306.2,1271.0,1304.3,1300.5,1304.3,1291.1,0.812500,-3.603465,-0.068182,0.309180
2024-08-01 11:20:00,1298.8,1300.0,1298.2,1299.3,3764,2024-08-01,1275.0,1306.2,1271.0,1304.3,1300.5,1304.3,1291.1,0.803977,-3.833474,-0.090909,0.329836
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.867925,-0.235294,-0.250000,0.482568


### No signal for a Long

In [18]:
data_test[(data_test.percent < 0.2) & (data_test.Predicts > 0.2)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-02 11:05:00,1133.9,1134.2,1132.0,1132.9,5737,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.175926,-1.498325,0.116279,0.591122
2024-01-02 11:10:00,1132.8,1133.3,1131.7,1132.8,4397,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.166667,-1.586462,0.104651,0.406485
2024-01-02 11:15:00,1132.7,1132.8,1131.4,1132.0,3338,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.092593,-2.291556,0.011628,0.420932
2024-01-02 11:20:00,1132.0,1133.0,1131.0,1132.7,5447,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.157407,-1.674599,0.093023,0.406564
2024-01-02 11:25:00,1132.8,1133.2,1132.5,1132.9,2369,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.175926,-1.498325,0.116279,0.591122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 11:10:00,1259.2,1260.0,1256.3,1259.7,7626,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.160377,-12.000000,-0.676136,0.571361
2024-08-02 11:20:00,1260.7,1261.1,1260.0,1260.5,2852,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.198113,-11.372549,-0.653409,0.454925
2024-08-02 13:25:00,1261.0,1261.3,1259.0,1260.1,6467,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.179245,-11.686275,-0.664773,0.375011


### Wrong(switch) Long-Short signal

In [21]:
data_test[((data_test.percent > 0.8) & (data_test.Predicts < 0.2)) | (data_test.percent < 0.2) & (data_test.Predicts > 0.8)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-19 09:00:00,1181.0,1181.4,1180.4,1180.7,8476,2024-01-19,1190.0,1190.0,1180.4,1172.8,1176.0,1177.8,1165.8,0.031250,6.736016,0.391667,0.800742
2024-03-12 09:00:00,1232.9,1234.0,1232.3,1233.2,4553,2024-03-12,1239.8,1243.0,1232.0,1231.0,1246.0,1254.5,1231.0,0.109091,1.787165,-0.544681,0.819899
2024-06-03 09:30:00,1282.8,1283.4,1282.1,1282.1,3704,2024-06-03,1291.5,1298.4,1280.7,1270.0,1272.1,1275.2,1266.7,0.079096,9.527559,1.176471,0.811371


### Correct Long-Short signals

In [19]:
data_test[((data_test.percent > 0.8) & (data_test.Predicts > 0.8)) | (data_test.percent < 0.2) & (data_test.Predicts < 0.2)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-31 14:25:00,1171.1,1171.7,1165.7,1167.0,11507,2024-01-31,1171.0,1188.5,1165.7,1186.0,1185.0,1187.2,1182.7,0.057018,-16.020236,-4.000000,0.089371
2024-01-31 14:30:00,1166.2,1166.2,1166.1,1166.1,102,2024-01-31,1171.0,1188.5,1165.7,1186.0,1185.0,1187.2,1182.7,0.017544,-16.779089,-4.200000,0.096521
2024-02-05 14:00:00,1202.9,1211.0,1202.6,1206.0,13553,2024-02-05,1202.5,1211.0,1181.3,1181.1,1178.7,1182.3,1174.9,0.831650,21.082042,3.689189,0.932672
2024-02-05 14:10:00,1204.8,1208.4,1204.5,1208.4,7767,2024-02-05,1202.5,1211.0,1181.3,1181.1,1178.7,1182.3,1174.9,0.912458,23.114046,4.013514,0.881196
2024-02-05 14:15:00,1208.6,1208.6,1205.7,1206.6,6371,2024-02-05,1202.5,1211.0,1181.3,1181.1,1178.7,1182.3,1174.9,0.851852,21.590043,3.770270,0.932672
2024-02-07 13:55:00,1215.1,1217.9,1215.1,1217.2,4333,2024-02-07,1219.0,1219.8,1205.3,1203.9,1203.2,1205.8,1200.8,0.820690,11.047429,2.800000,0.826520
2024-02-07 14:00:00,1217.5,1218.7,1216.6,1217.5,4381,2024-02-07,1219.0,1219.8,1205.3,1203.9,1203.2,1205.8,1200.8,0.841379,11.296619,2.860000,0.813608
2024-02-07 14:05:00,1217.7,1219.4,1217.5,1218.7,5603,2024-02-07,1219.0,1219.8,1205.3,1203.9,1203.2,1205.8,1200.8,0.924138,12.293380,3.100000,0.910065
2024-02-07 14:10:00,1218.9,1219.8,1217.8,1218.0,4764,2024-02-07,1219.0,1219.8,1205.3,1203.9,1203.2,1205.8,1200.8,0.875862,11.711936,2.960000,0.897992
